# FEAT method

#### Importation of the librairies

In [120]:
import pandas as pd
import numpy as np

### List of the flights, according to Eurocontrol

In [121]:
# !pip install nb_black

In [122]:
# %load_ext nb_black
# réglé par défaut à 120 caractères par colonne

In [134]:
df_flights = (
    pd.read_csv("flights_eurocontrol.csv")
    .rename(columns={"Actual Distance Flown (nm)": "Distance_nm"})
    .eval("Distance_km= Distance_nm*1.852")
)

df_eurocontrol = df_flights.query(
    '(ADEP.str.startswith("E") | ADEP.str.startswith("L")) & (ADES.str.startswith("E") | ADES.str.startswith("L"))',engine='python'
)

### List of the intern flights

In [124]:
df_flights = (
    pd.read_csv("flights_eurocontrol.csv")
    .rename(columns={"Actual Distance Flown (nm)": "Distance_nm"})
    .eval("Distance_km= Distance_nm*1.852")
)


df_eurocontrol = df_flights.query(
    '(ADEP.str.startswith("E") | ADEP.str.startswith("L")) & (ADES.str.startswith("E") | ADES.str.startswith("L"))', engine='python'
).drop(
    columns=[
        "FILED OFF BLOCK TIME",
        "FILED ARRIVAL TIME",
        "ACTUAL OFF BLOCK TIME",
        "ACTUAL ARRIVAL TIME",
        "AC Operator",
        "AC Registration",
    ]
)

df_eurocontrol.head()

,ECTRL ID,ADEP,ADEP Latitude,ADEP Longitude,ADES,ADES Latitude,ADES Longitude,AC Type,ICAO Flight Type,STATFOR Market Segment,Requested FL,Distance_nm,Distance_km
0,242070088,EDDP,51.42389,12.23639,EGMC,51.57028,0.69333,B734,S,All-Cargo,320.0,435,805.620
1,242072146,EDDK,50.86583,7.14278,LFLL,45.72556,5.08111,B734,S,All-Cargo,240.0,350,648.200
2,242074255,EPKT,50.47417,19.08000,EDDK,50.86583,7.14278,B734,S,All-Cargo,280.0,481,890.812
4,242075583,EDDV,52.46028,9.68361,EGNX,52.83111,-1.32778,B734,S,All-Cargo,340.0,416,770.432
5,242076646,EGTE,50.73417,-3.41389,EGNX,52.83111,-1.32778,B734,S,All-Cargo,210.0,169,312.988


In [125]:
df_eurocontrol.query("ADEP == 'EDDP' & ADES == 'EGNX'").groupby("AC Type").count().sum()

ECTRL ID                  210
ADEP                      210
ADEP Latitude             210
ADEP Longitude            210
ADES                      210
ADES Latitude             210
ADES Longitude            210
ICAO Flight Type          210
STATFOR Market Segment    210
Requested FL              210
Distance_nm               210
Distance_km               210
dtype: int64

In [126]:
df_eurocontrol["STATFOR Market Segment"].unique()

array(['All-Cargo', 'Lowcost', 'Mainline', 'Business Aviation',
       'Regional Aircraft', 'Charter', 'Other Types'], dtype=object)

In [127]:
df_eurocontrol["ICAO Flight Type"].unique()

array(['S', 'N'], dtype=object)

### List of the existing airports

In [128]:
df_airports = pd.read_csv("airports.csv")
df_airports.head(3)

,name,iata,icao,latitude,longitude,country,altitude,type,municipality
0,Honiara International Airport,HIR,AGGH,-9.428000,160.054993,Solomon Islands,28.0,large_airport,Honiara
1,Munda Airport,MUA,AGGM,-8.327970,157.263000,Solomon Islands,10.0,medium_airport,Munda
2,Nauru International Airport,INU,ANYN,-0.547458,166.919006,Nauru,22.0,medium_airport,Yaren District


In [129]:
df_airports.shape

(4407, 9)

In [130]:
df_airports_romania = df_airports.query("country == 'France'", engine='python')
df_airports_romania

,name,iata,icao,latitude,longitude,country,altitude,type,municipality
1935,Calais-Dunkerque Airport,CQF,LFAC,50.962101,1.954760,France,12.0,medium_airport,Marck
1936,Le Touquet-Côte d'Opale Airport,LTQ,LFAT,50.517399,1.620590,France,36.0,medium_airport,Le Touquet-Paris-Plage
1937,Agen-La Garenne Airport,AGF,LFBA,44.174702,0.590556,France,204.0,medium_airport,Agen/La Garenne
1938,Bordeaux-Mérignac Airport,BOD,LFBD,44.828300,-0.715556,France,162.0,large_airport,Bordeaux/Mérignac
1939,Bergerac-Roumanière Airport,EGC,LFBE,44.825298,0.518611,France,171.0,medium_airport,Bergerac/Roumanière
...,...,...,...,...,...,...,...,...,...
2035,Brive-Souillac,BVE,LFSL,45.039722,1.485556,France,1016.0,medium_airport,Brive la Gaillarde
2036,Nancy-Essey Airport,ENC,LFSN,48.692101,6.230460,France,751.0,medium_airport,Nancy/Essey
2037,Strasbourg Airport,SXB,LFST,48.538300,7.628230,France,505.0,medium_airport,Strasbourg
2038,Toulon-Hyères Airport,TLN,LFTH,43.097301,6.146030,France,7.0,medium_airport,Toulon/Hyères/Le Palyvestre


### FEAT estimation of the fuel consumption

The fuel consumption is determined from the formula :
$$F_i = \alpha_i \cdot d_{gc}^2 + \beta_i \cdot d_{gc} + \gamma_i$$
Where $\alpha_i$, $\beta_i$ and $\gamma_i$ are parameters that are calculated from the OAG dataset and can be find in the following github page : https://github.com/kwdseymour/FEAT 

In [168]:
# Importation of the coefficients and calculus of the fuel consumption
df_coeff = pd.read_csv("feat_coefficients.csv")
df_data = pd.merge(
    df_eurocontrol, df_coeff, left_on="AC Type", right_on="ac_code_icao"
).eval(
    "Fuel_consumption_kg= Distance_km**2 * reduced_fuel_a1 + Distance_km *reduced_fuel_a2 + reduced_fuel_intercept"
)
df_coeff

,Unnamed: 0,ac_code_icao,e_type,wake,reduced_fuel_a1,reduced_fuel_a2,reduced_fuel_intercept,reduced_fuel_score,reduced_sample_size
0,0,A140,Turboprop,M,0.000157,1.298278,160.624472,0.999946,25
1,1,A148,Jet,M,0.000064,1.757268,971.035823,0.999795,25
2,2,A158,Jet,M,0.000033,2.566019,799.939398,0.999353,24
3,3,A20N,Jet,M,0.000057,2.382220,955.977183,0.999824,25
4,4,A21N,Jet,M,0.000063,2.723659,1121.938176,0.999939,25
...,...,...,...,...,...,...,...,...,...
128,128,T154,Jet,M,-0.000037,6.335285,1809.615119,0.999395,25
129,129,T204,Jet,M,0.000152,3.766157,1780.359925,0.999934,25
130,130,Y12,Turboprop,L,0.000007,0.470331,37.756806,0.999926,25
131,131,YK40,Jet,M,-0.000155,2.482920,506.996684,0.999885,25


In [155]:
#FE Turboprop pas fou

df_emissions = pd.concat(
    [
        df_data.query("e_type == 'Jet'").eval("Emissions = Fuel_consumption_kg*3.16"),
        df_data.query("e_type == 'Piston'").eval(
            "Emissions = Fuel_consumption_kg*3.10"
        ),
        df_data.query("e_type == 'Turboprop'").eval(
            "Emissions = Fuel_consumption_kg*3.16"
        )
    ]
)

round(df_emissions["Emissions"].sum() / 1000 / 1000 / 1000 * 12, 2) #MtCO2
df_emissions.head()



,ECTRL ID,ADEP,ADEP Latitude,ADEP Longitude,ADES,ADES Latitude,ADES Longitude,AC Type,ICAO Flight Type,STATFOR Market Segment,...,ac_code_icao,e_type,wake,reduced_fuel_a1,reduced_fuel_a2,reduced_fuel_intercept,reduced_fuel_score,reduced_sample_size,Fuel_consumption_kg,Emissions
0,242070088,EDDP,51.42389,12.23639,EGMC,51.57028,0.69333,B734,S,All-Cargo,...,B734,Jet,M,0.000104,3.210174,1167.77325,0.999983,25,3821.571627,12076.166342
1,242072146,EDDK,50.86583,7.14278,LFLL,45.72556,5.08111,B734,S,All-Cargo,...,B734,Jet,M,0.000104,3.210174,1167.77325,0.999983,25,3292.382381,10403.928325
2,242074255,EPKT,50.47417,19.08000,EDDK,50.86583,7.14278,B734,S,All-Cargo,...,B734,Jet,M,0.000104,3.210174,1167.77325,0.999983,25,4110.109742,12987.946785
3,242075583,EDDV,52.46028,9.68361,EGNX,52.83111,-1.32778,B734,S,All-Cargo,...,B734,Jet,M,0.000104,3.210174,1167.77325,0.999983,25,3702.834159,11700.955942
4,242076646,EGTE,50.73417,-3.41389,EGNX,52.83111,-1.32778,B734,S,All-Cargo,...,B734,Jet,M,0.000104,3.210174,1167.77325,0.999983,25,2182.725192,6897.411607


In [169]:
def function_fuel(plane_icao, distance, df_coeff = df_coeff):
    fuel = df_coeff.query("ac_code_icao == @plane_icao").eval("reduced_fuel_a1*@distance**2 + reduced_fuel_a2*@distance + reduced_fuel_intercept")
    return(fuel)

In [170]:
function_fuel('B734', 100)

38    1489.832475
dtype: float64

### Estimer le nombre total de passagers par ligne 

In [139]:
planes_eurocontrol = df_eurocontrol["AC Type"].unique()
planes_coeff = df_coeff.ac_code_icao.unique()
intersection = [x for x in planes_eurocontrol if x in planes_coeff]
intersection

['B734',
 'AT43',
 'A21N',
 'B752',
 'B738',
 'C25A',
 'A306',
 'SF34',
 'ATP',
 'B762',
 'B733',
 'CRJ9',
 'B463',
 'E120',
 'AT72',
 'F406',
 'A332',
 'B763',
 'E195',
 'B739',
 'BE20',
 'CRJ2',
 'SW4',
 'A321',
 'A20N',
 'A333',
 'A320',
 'B77L',
 'B748',
 'B77W',
 'BCS3',
 'DH8A',
 'B744',
 'E75S',
 'B789',
 'DH8D',
 'E55P',
 'CRJX',
 'DH8B',
 'A319',
 'B737',
 'E190',
 'A318',
 'E75L',
 'JS31',
 'BE40',
 'B190',
 'DH8C',
 'SB20',
 'F50',
 'A359',
 'C680',
 'E145',
 'E170',
 'PC12',
 'A343',
 'J328',
 'JS41',
 'E135',
 'C208',
 'L410',
 'D228',
 'DHC6',
 'A35K',
 'D328',
 'B772',
 'GLF5',
 'BCS1',
 'A310',
 'B735',
 'GLF4',
 'F100',
 'B788',
 'CRJ7',
 'A346',
 'BN2P',
 'PA31',
 'A339',
 'JS32',
 'B712',
 'B736',
 'RJ1H',
 'MD83',
 'A140',
 'RJ85',
 'SU95',
 'MD82',
 'T204',
 'A388',
 'C172']

In [154]:
df_pax_capacity = pd.DataFrame(pd.read_csv("list-planes-capacity.csv", sep=";"))
df_pax_capacity


,code_icao,min,max
0,A140,52,54
1,A20N,150,180
2,A21N,180,244
3,A306,200,360
4,A310,191,279
...,...,...,...
85,SB20,50,58
86,SF34,30,37
87,SU95,75,98
88,SW4,12,19


### Estimer les émissions par passager et par avion (pré-traitement)

##### We test the number of flight that are in Europe

Utiliser FEAT pour tous les vols
OpenAP : plus de degrés de liberté (ex infuence du taux de remplissage, influence de la masse) --> dépend de nos critères d'optimisation. 

### Équilibre fréquence/capacité

Déterminer la fréquence nécessaire pour chaque avion en fonction du nombre de passagers :

Pour chaque avion $a$, on définit :

fréquence : $f_a = \frac{n_{vol_a}}{temps}$

débit de passagers : $d = \frac{n_{passagers}}{temps}$

capacité de l'avion : $c_a = \frac{n_{passagers}}{vol}$

On a donc $f_a = \frac{d}{c_a}$, ce qui nous permet de relier pour chaque avion la fréquence à laquelle il doit circuler pour respecter un débit de passagers donné.

On discrétise par heure : $t \in \{0, ..., 23\}$

Ce qu'on veut minimiser, c'est 
$$\sum_a FE_a * fuel_a * n_{vol_a} =\sum_a \sum_h FE_a * fuel_a * f_a(h)

= \sum_a \sum_h FE_a * fuel_a * \frac{d}{c_a}$$
sous les contraintes
$$ f_a > UNE VALEUR À DÉFINIR $$


## Optimization

Probleme 1 : Trouver le couple (typde d'avion, fréquence de vol) qui minimise les émissions par jour sur la ligne Orly - Toulouse en respectant une fréquence minimale de 15 vols par jours et en transportant tout les passagers.
2Prèoblèe me à 2 variables

Données :

In [119]:
p_an = 2.3e6 #nombre de passagers annuels sur la ligne Orly-Toulouse 
p_jour = round(p_an/365) #nb de passagers journaliers
fv_min = 15 #fréquence minimale de vol par jour
d = df_eurocontrol.query("ADEP == 'LFBO' & ADES == 'LFPO'").Distance_km.mean()



Problème 2 : Trouver la liste des différents types d'avion qui minimise les émissions par jour sur la ligne Orly - Toulouse en respectant une fréquence minimale de 15 vols par jours et en transportant tout les passagers.
Précision : on considère 50 vols max sur la journée, donc 50 variables

1st : pas de train si trajet direct 
capacité moyenne pour avoir le nombre total de passagers 
modélisation du probleme pour la semaine pro